In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import pandas as pd
from scipy import ndimage
from IPython.display import Image
import random
from pathlib import Path
import cv2
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
def gen_images(img):
  image= cv2.imread(img)
  image = cv2.resize(image, (128,128))
  return image

In [3]:
data = []
labels = []
dir = '../input/nnfl-2021-assignment-1/fire_videos/train'
df = pd.read_csv("../input/nnfl-2021-assignment-1/fire_videos/train.csv")
for i in range(len(df.File)):
  if i%11 == 0:
    imagepath = os.path.join(dir,df.File[i])
    data.append(gen_images(imagepath))
    if df.True_Label[i] == 'fire':
      labels.append([1,0])
    else:
      labels.append([0,1])
  else:
    continue

In [4]:
print(len(data))
print(len(labels))
data = np.array(data, dtype="float")/255.0
labels = np.array(labels)
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.10, random_state=42)

6268
6268


In [7]:
from tensorflow.keras.layers import Dropout
model = models.Sequential()
model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(2, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 124, 124, 64)      4864      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 256)       295168    
_________________________________________________________________
flatten_1 (Flatten)          (None, 200704)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)               

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=20, batch_size= 64)

2021-10-28 17:05:15.503888: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1109065728 exceeds 10% of free system memory.
2021-10-28 17:05:16.664682: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1109065728 exceeds 10% of free system memory.


Epoch 1/20
89/89 [==============================] - 5s 45ms/step - loss: 0.5407 - accuracy: 0.7406 - val_loss: 0.3592 - val_accuracy: 0.8341
Epoch 2/20
89/89 [==============================] - 3s 37ms/step - loss: 0.4202 - accuracy: 0.7851 - val_loss: 0.2548 - val_accuracy: 0.8740
Epoch 3/20
89/89 [==============================] - 3s 38ms/step - loss: 0.3558 - accuracy: 0.7986 - val_loss: 0.2339 - val_accuracy: 0.8325
Epoch 4/20
89/89 [==============================] - 3s 37ms/step - loss: 0.3613 - accuracy: 0.7894 - val_loss: 0.2469 - val_accuracy: 0.8453
Epoch 5/20
89/89 [==============================] - 3s 37ms/step - loss: 0.3583 - accuracy: 0.7990 - val_loss: 0.1721 - val_accuracy: 0.8836
Epoch 6/20
89/89 [==============================] - 3s 39ms/step - loss: 0.2989 - accuracy: 0.8107 - val_loss: 0.1635 - val_accuracy: 0.8772
Epoch 7/20
89/89 [==============================] - 3s 37ms/step - loss: 0.2696 - accuracy: 0.8420 - val_loss: 0.1257 - val_accuracy: 0.9601
Epoch 8/20
89

In [9]:
test_data = []
dir = '../input/nnfl-2021-assignment-1/fire_videos/test'
df1 = pd.read_csv("../input/nnfl-2021-assignment-1/fire_videos/test.csv")
for i in range(len(df1.File)):
    imagepath = os.path.join(dir,df1.File[i])
    test_data.append(gen_images(imagepath))
test_data = np.array(test_data, dtype="float")/255.0

In [10]:
test_labels=[]
test_labels = model.predict(test_data)

2021-10-28 17:07:15.325783: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 554434560 exceeds 10% of free system memory.


In [11]:
class_prediction = []
for i in range(len(test_labels)):
    class_prediction.append(np.argmax(test_labels[i]))

In [12]:
pred = []
K = 0
for i in class_prediction :
    if i == 1:
        pred.append('not_fire')
        K = K + 1
    else:
        pred.append('fire')
print(K)

2259


In [13]:
dict = {'File': df1.File, 'Label': pred} 
     
df1 = pd.DataFrame(dict)
  
# saving the dataframe
df1.to_csv('./subfile35.csv',index=False)